# HABITABILITY OF EXOPLANETS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr

# Importing file and displaying column

In [ ]:
file=pd.read_csv(r'exoplanet_dataset.csv')
for column in file.columns:
    print (column)
p_name=file['P_NAME']

# Using stellar luminosity and orbital distance

In [ ]:
s_luminosity = file['S_LUMINOSITY']  # Star luminosity in solar units
s_distance = file['S_DISTANCE']  # Planet's distance from its star in AU

# Create lists to store classified planets
habitable_1 = []
partially_1 = []
not_habitable_1 = []

# Define habitable zone calculation using luminosity
def calculate_habitable_zone(luminosity):
    # Inner and outer habitable zone boundaries
    inner_hz = np.sqrt(luminosity / 1.1)
    outer_hz = np.sqrt(luminosity / 0.53)
    return inner_hz, outer_hz

# Classify planets based on their distance from the star and luminosity
for i in range(len(p_name)):
    luminosity = s_luminosity[i]
    distance = s_distance[i]

    # Calculate habitable zone for the star
    inner_hz, outer_hz = calculate_habitable_zone(luminosity)
    
    # Check if the planet's distance falls within the habitable zone
    if inner_hz <= distance <= outer_hz:
        habitable_1.append(p_name[i])  # Planet is in the habitable zone
    elif distance < inner_hz:  # Planet is too close, likely too hot
        if distance > inner_hz * 0.75:  # Partially habitable zone (close to the inner boundary)
            partially_1.append(p_name[i])
        else:
            not_habitable_1.append(p_name[i])
    elif distance > outer_hz:  # Planet is too far, likely too cold
        if distance < outer_hz * 1.25:  # Partially habitable zone (close to the outer boundary)
            partially_1.append(p_name[i])
        else:
            not_habitable_1.append(p_name[i])

# Print the classified planets
print("Habitable Planets:")
print(len(habitable_1))

print("\nPartially Habitable Planets:")
print(len(partially_1))

print("\nNot Habitable Planets:")
print(len(not_habitable_1))


# Stellar tempareture and type of star

In [ ]:
s_luminosity = file['S_LUMINOSITY']  # Star luminosity in solar units
s_temperature = file['S_TEMPERATURE']  # Star temperature in Kelvin
s_distance = file['S_DISTANCE']  # Planet's distance from its star in AU
s_type = file['S_TYPE']  # Type of the star (G-type, K-type, etc.)

# Create lists to store classified planets
habitable_2 = []
partially_2 = []
not_habitable_2 = []

def get_star_properties(star_type):
    # Define properties based on star type (luminosity in solar units, temperature in Kelvin)
    if star_type == 'G':
        # G-type stars (like our Sun)
        luminosity = 1.0
        temperature = 5778
    elif star_type == 'K':
        # K-type stars
        luminosity = 0.5
        temperature = 5000
    elif star_type == 'M':
        # M-type stars (red dwarfs)
        luminosity = 0.1
        temperature = 3500
    elif star_type == 'B':
        # B-type stars
        luminosity = 10.0
        temperature = 20000
    else:
        # Default to Sun-like properties if the type is unknown
        luminosity = 1.0
        temperature = 5778
    
    return luminosity, temperature

# Define habitable zone calculation using luminosity and temperature
def calculate_habitable_zone(luminosity, temperature):
    # Adjust habitable zone based on luminosity and temperature
    inner_hz = np.sqrt(luminosity / (temperature**4)) * 1.0  # Inner habitable zone (closer for cooler stars)
    outer_hz = np.sqrt(luminosity / (temperature**4)) * 1.25  # Outer habitable zone (farther for hotter stars)
    return inner_hz, outer_hz

# Classify planets based on their distance from the star and luminosity & temperature
for i in range(len(p_name)):
    star_type = s_type[i]
    
    # Get the star's luminosity and temperature based on its type
    luminosity, temperature = get_star_properties(star_type)
    
    distance = s_distance[i]

    # Calculate habitable zone for the star
    inner_hz, outer_hz = calculate_habitable_zone(luminosity, temperature)
    
    # Check if the planet's distance falls within the habitable zone
    if inner_hz <= distance <= outer_hz:
        habitable_2.append(p_name[i])  # Planet is in the habitable zone
    elif distance < inner_hz:  # Planet is too close, likely too hot
        if distance > inner_hz * 0.75:  # Partially habitable zone (close to the inner boundary)
            partially_2.append(p_name[i])
        else:
            not_habitable_2.append(p_name[i])
    elif distance > outer_hz:  # Planet is too far, likely too cold
        if distance < outer_hz * 1.25:  # Partially habitable zone (close to the outer boundary)
            partially_2.append(p_name[i])
        else:
            not_habitable_2.append(p_name[i])

# Print the classified planets
print("Habitable Planets:")
print(len(habitable_2))

print("\nPartially Habitable Planets:")
print(len(partially_2))

print("\nNot Habitable Planets:")
print(len(not_habitable_2))



# Surface Gravity

In [ ]:
p_gravity = file['P_GRAVITY']

min_gravity = 3.0    # Minimum gravity to retain an atmosphere
max_gravity = 25.0   # Above this, the planet is likely a gas giant
lower_border = 1.5   # Borderline low gravity (may struggle to hold atmosphere)
upper_border = 35.0  # Borderline high gravity (dense atmosphere, but not gas giant)

# Classify planets
habitable_3 = file[(p_gravity >= min_gravity) & (p_gravity <= max_gravity)]
partially_3 = file[((p_gravity >= lower_border) & (p_gravity < min_gravity)) | 
                           ((p_gravity > max_gravity) & (p_gravity <= upper_border))]
not_habitable_3 = file[(p_gravity < lower_border) | (p_gravity > upper_border)]

# Print results
print( len(habitable_3))
print( len(partially_3))
print(len(not_habitable_3))


# Surface temperature

In [ ]:
p_temp_surf = file['P_TEMP_SURF']

min_habitable_temp = 273  # 0°C (water freezes)
max_habitable_temp = 373  # 100°C (water boils)

lower_border_temp = 200   # Partially habitable (cold, but could have subsurface liquid water,europa)
upper_border_temp = 450   # Partially habitable (very hot, but could support extremophiles)

# Classify planets based on temperature
habitable_4= file[(p_temp_surf >= min_habitable_temp) & (p_temp_surf <= max_habitable_temp)]
partially_4 = file[((p_temp_surf >= lower_border_temp) & (p_temp_surf < min_habitable_temp)) | 
                                ((p_temp_surf > max_habitable_temp) & (p_temp_surf <= upper_border_temp))]
not_habitable_4 = file[(p_temp_surf < lower_border_temp) | (p_temp_surf > upper_border_temp)]

# Print results
print(len(habitable_4))
print(len(partially_4))
print(len(not_habitable_4))


# Desity of the planet

In [ ]:
p_density = file['P_DENSITY']

min_habitable_density = 3.0   # g/cm³ (low-end rocky planets)
max_habitable_density = 7.0   # g/cm³ (high-end rocky planets)

lower_border_density = 1.5    # Borderline low-density (could be water-rich or icy planets)
upper_border_density = 10.0   # Borderline high-density (very massive rocky planets or iron-rich worlds)

# Classify planets based on density
habitable_4 = file[(p_density >= min_habitable_density) & (p_density <= max_habitable_density)]
partially_4 = file[((p_density >= lower_border_density) & (p_density < min_habitable_density)) | 
                                   ((p_density > max_habitable_density) & (p_density <= upper_border_density))]
not_habitable_4 = file[(p_density < lower_border_density) | (p_density > upper_border_density)]

# Print results
print(len(habitable_4))
print( len(partially_4))
print(len(not_habitable_4))


# Albedo(reflectivity)

In [ ]:
sigma = 5.67 * (10**-8)  # Stefan-Boltzmann constant in W/m²K⁴

# Extract data from the CSV file
luminosity = file['S_LUMINOSITY']  # Star's luminosity (in solar units)
distance = file['P_DISTANCE']  # Planet's distance from the star (in AU)
p_temp_surf = file['P_TEMP_SURF']  # Planet's surface temperature (in Kelvin)

# Convert luminosity from solar units to Watts
luminosity = luminosity * 3.846 * (10**26)  # 1 solar luminosity = 3.846 * 10^26 W

# Convert distance from AU to meters (1 AU = 1.496 * 10^11 meters)
distance = distance * 1.496 * (10**11)

# Calculate effective temperature (T_eff) in Kelvin
T_eff = (luminosity / (16 * np.pi * distance**2 * sigma))**(1/4)

# Estimate albedo using the formula for radiative equilibrium
# Assuming perfect blackbody (emissivity ~ 1), use the relation to calculate albedo
albedo = 1 - (T_eff / p_temp_surf)**4

# Ensure albedo is within [0, 1] range (as albedo cannot be greater than 1 or negative)
albedo = np.clip(albedo, 0, 1)

# Print or save results
file['P_ALBEDO'] = albedo  # Add calculated albedo to the dataframe
print("Calculated albedo for planets:")
print(file[['P_NAME', 'P_ALBEDO']])

# Handle NaN values in albedo before classification
file = file.dropna(subset=['P_ALBEDO'])  # Remove rows with NaN albedo values

# Classify planets based on albedo values
habitable_6 = file[(file['P_ALBEDO'] >= 0.2) & (file['P_ALBEDO'] <= 0.4)]
partially_6 = file[((file['P_ALBEDO'] >= 0.15) & (file['P_ALBEDO'] < 0.2)) | 
                                  ((file['P_ALBEDO'] > 0.4) & (file['P_ALBEDO'] <= 0.6))]
not_habitable_6 = file[(file['P_ALBEDO'] < 0.15) | (file['P_ALBEDO'] > 0.6)]

# Print classification results
print(len(habitable_6))
print(len(partially_6))
print(len(not_habitable_6))


# THE REAL DEAL

# Mass radius relationship and internal composition

In [ ]:
df = pd.read_csv("exoplanet_dataset.csv")
# converting mass and radius wrt earth.
# Define conditions where mass conversion is needed
terran_fix = (df["P_TYPE"] == "Terran") & (~df["P_MASS"].between(0.5, 5))
superterran_fix = (df["P_TYPE"] == "Superterran") & (~df["P_MASS"].between(1.9, 10))
neptunian_fix = (df["P_TYPE"] == "Neptunian") & (~df["P_MASS"].between(5, 10))
jovian_fix = (df["P_TYPE"] == "Jovian") & (df["P_MASS"] < 10)
# Combine all conversion conditions
mass_fix = terran_fix | superterran_fix | neptunian_fix | jovian_fix
# Convert mass if condition fails
df.loc[mass_fix, "P_MASS"] *= 317.8
# Convert radius ONLY if mass was converted
df.loc[mass_fix, "P_RADIUS"] *= 11.2
# Save cleaned data
df.to_csv("new.csv", index=False)

df["P_DENSITY"] = (3 * df["P_MASS"]) / (4 * np.pi * (df["P_RADIUS"] ** 3))
df.to_csv("new.csv", index=False)

In [ ]:
df = pd.read_csv("new.csv")

# Ensure no zero or negative values before taking log
df = df[(df["P_MASS"] > 0) & (df["P_RADIUS"] > 0)]

# Log-transform mass and radius
df["log_M"] = np.log10(df["P_MASS"])
df["log_R"] = np.log10(df["P_RADIUS"])

# Scatter Plot
plt.figure(figsize=(8, 6))
scatter = sns.scatterplot(
    x=df["log_M"], 
    y=df["log_R"], 
    hue=df["P_TEMP_EQUIL"],  
    palette="coolwarm",  
    alpha=0.7
)

# Fix for the colorbar
norm = plt.Normalize(df["P_TEMP_EQUIL"].min(), df["P_TEMP_EQUIL"].max())  
sm = plt.cm.ScalarMappable(cmap="coolwarm", norm=norm)
cbar = plt.colorbar(sm, ax=plt.gca())  # Link colorbar to the current axis
cbar.set_label("Equilibrium Temperature (K)")  

plt.xlabel("log(Mass) [Earth Masses]")
plt.ylabel("log(Radius) [Earth Radii]")
plt.title("Mass-Radius Relation (Log-Log Scale)")
plt.grid(True, linestyle="--", linewidth=0.5)
plt.show()


### STEP 3: Fit Power-Law Model R = k * M^n
def mass_radius_relation(M, k, n):
    return k * (M ** n)

# Fit using curve_fit (nonlinear least squares)
popt, pcov = curve_fit(mass_radius_relation, df["P_MASS"], df["P_RADIUS"], p0=[1, 0.3])
k_fit, n_fit = popt

# STEP 4: Linear Regression on log-transformed data
X = df["log_M"].values.reshape(-1, 1)
y = df["log_R"].values.reshape(-1, 1)
lin_reg = LinearRegression().fit(X, y)
n_linear = lin_reg.coef_[0][0]  # Slope = n
k_linear = 10 ** lin_reg.intercept_[0]  # Intercept in log space -> k

# Plot fitted line
plt.figure(figsize=(8, 6))
plt.scatter(df["log_M"], df["log_R"], label="Data", alpha=0.6, color="blue")
plt.plot(df["log_M"], lin_reg.predict(X), color="red", label=f"Fit: logR = {np.log10(k_linear):.2f} + {n_linear:.2f} logM")
plt.xlabel("log(Mass)")
plt.ylabel("log(Radius)")
plt.title("Linear Fit (Power-Law Model)")
plt.legend()
plt.grid(True, linestyle="--")
plt.show()

### STEP 5: Interpretation
print(f"Best-fit parameters: k = {k_fit:.2f}, n (curve_fit) = {n_fit:.2f}")
print(f"Best-fit parameters: k = {k_linear:.2f}, n (linear regression) = {n_linear:.2f}")

# Compare n to theoretical models
if n_fit < 0.2:
    print("Planets are likely gas giants (compressibility effects).")
elif 0.2 <= n_fit < 0.27:
    print("Planets are likely iron-rich (dense rocky composition).")
elif 0.27 <= n_fit < 0.3:
    print("Planets are likely silicate (Earth-like rocky planets).")
else:
    print("Planets may be water-rich or mixed composition.")

In [ ]:
def mass_radius_relation(M, k, n):
    return k * (M ** n)

# Planet categories
types = ["Terran", "Superterran", "Neptunian", "Jovian"]
colors = ["blue", "green", "orange", "red"]

plt.figure(figsize=(8, 6))

fit_results = {}

for planet_type, color in zip(types, colors):
    subset = df[df["P_TYPE"] == planet_type]
    
    if not subset.empty:
        # Nonlinear Curve Fit
        popt, _ = curve_fit(mass_radius_relation, subset["P_MASS"], subset["P_RADIUS"], p0=[1, 0.3])
        k_fit, n_fit = popt
        fit_results[planet_type] = (k_fit, n_fit)
        
        # Linear Regression
        X = subset["log_M"].values.reshape(-1, 1)
        y = subset["log_R"].values.reshape(-1, 1)
        lin_reg = LinearRegression().fit(X, y)
        n_linear = lin_reg.coef_[0][0]
        k_linear = 10 ** lin_reg.intercept_[0]
        
        # Scatter Plot
        plt.scatter(subset["log_M"], subset["log_R"], label=f"{planet_type} (n={n_fit:.2f})", alpha=0.6, color=color)
        
        # Plot Fit
        mass_range = np.linspace(subset["log_M"].min(), subset["log_M"].max(), 100)
        plt.plot(mass_range, np.log10(mass_radius_relation(10**mass_range, k_fit, n_fit)), color=color)

plt.xlabel("log(Mass) [Earth Masses]")
plt.ylabel("log(Radius) [Earth Radii]")
plt.title("Mass-Radius Relation by Planet Type")
plt.legend()
plt.grid(True, linestyle="--")
plt.show()

# Print Results
for planet_type, (k_fit, n_fit) in fit_results.items():
    print(f"{planet_type}: k = {k_fit:.2f}, n = {n_fit:.2f}")

# Orbital Eccentricity vs. Habitability:

In [ ]:
df = df[(df["P_ECCENTRICITY"] >= 0) & (df["P_TEMP_SURF"] > 0)]

df["P_ECCENTRICITY"].fillna(df["P_ECCENTRICITY"].mean(), inplace=True)
df["P_TEMP_SURF"].fillna(df["P_TEMP_SURF"].mean(), inplace=True)

# Scatter Plot: Eccentricity vs. Surface Temperature
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df["P_ECCENTRICITY"], y=df["P_TEMP_SURF"], hue=df["P_HABITABLE"], palette="coolwarm", alpha=0.7)
plt.xlabel("Orbital Eccentricity")
plt.ylabel("Surface Temperature (K)")
plt.title("Orbital Eccentricity vs. Surface Temperature")
plt.grid(True, linestyle="--")
plt.legend(title="Habitability")
plt.show()

# Pearson Correlation: Eccentricity vs. Temperature Variations
corr, p_value = pearsonr(df["P_ECCENTRICITY"], df["P_TEMP_SURF"])
print(f"Pearson Correlation: {corr:.3f}, p-value: {p_value:.3f}")

# Histogram: Habitability distribution vs. Eccentricity
plt.figure(figsize=(8, 6))
sns.histplot(df, x="P_ECCENTRICITY", hue=df["P_HABITABLE"], bins=30, kde=True, palette="coolwarm", alpha=0.7)
plt.xlabel("Orbital Eccentricity")
plt.ylabel("Count")
plt.title("Habitability vs. Orbital Eccentricity")
plt.grid(True)
plt.show()

# Interpretation of results
if abs(corr) > 0.5 and p_value < 0.05:
    print("Strong correlation detected between eccentricity and surface temperature variations.")
    if corr > 0:
        print("Higher eccentricity leads to larger temperature swings, reducing habitability.")
    else:
        print("Higher eccentricity stabilizes temperature, potentially improving habitability.")
elif 0.2 < abs(corr) <= 0.5:
    print("Moderate correlation detected. Eccentricity may influence habitability, but other factors also play a role.")
else:
    print("Weak or no correlation found. Habitability may depend more on other factors like atmosphere or star type.")


In [ ]:
# Replace missing values with column mean
df["P_ECCENTRICITY"].fillna(df["P_ECCENTRICITY"].mean(), inplace=True)
df["P_TEMP_SURF"].fillna(df["P_TEMP_SURF"].mean(), inplace=True)
df["P_FLUX"].fillna(df["P_FLUX"].mean(), inplace=True)

# Compute Pearson correlations
corr_ecc_temp, p_ecc_temp = pearsonr(df["P_ECCENTRICITY"], df["P_TEMP_SURF"])
corr_flux_temp, p_flux_temp = pearsonr(df["P_FLUX"], df["P_TEMP_SURF"])

print(f"Pearson Correlation (Eccentricity vs. Surface Temp): {corr_ecc_temp:.3f}, p-value: {p_ecc_temp:.3f}")
print(f"Pearson Correlation (Flux vs. Surface Temp): {corr_flux_temp:.3f}, p-value: {p_flux_temp:.3f}")

# Scatter plot: Eccentricity vs. Temperature, colored by star type
plt.figure(figsize=(8,6))
sns.scatterplot(x=df["P_ECCENTRICITY"], y=df["P_TEMP_SURF"], hue=df["S_TYPE"], palette="viridis", alpha=0.7)
plt.xlabel("Orbital Eccentricity")
plt.ylabel("Surface Temperature (K)")
plt.title("Eccentricity vs. Surface Temperature (Colored by Star Type)")
plt.legend(title="Star Type")
plt.grid(True)
plt.show()

# Scatter plot: Flux vs. Temperature
plt.figure(figsize=(8,6))
sns.scatterplot(x=df["P_FLUX"], y=df["P_TEMP_SURF"], hue=df["S_TYPE"], palette="coolwarm", alpha=0.7)
plt.xlabel("Stellar Flux")
plt.ylabel("Surface Temperature (K)")
plt.title("Flux vs. Surface Temperature (Colored by Star Type)")
plt.legend(title="Star Type")
plt.grid(True)
plt.xscale("log")  # Log scale helps with large flux range
plt.show()

# Multi-variable regression: Predicting surface temperature
X = df[["P_ECCENTRICITY", "P_FLUX"]].copy()

# Log-transform flux (since flux values vary across large magnitudes)
X["P_FLUX"] = np.log10(X["P_FLUX"])

y = df["P_TEMP_SURF"]

# Fit regression model
reg = LinearRegression().fit(X, y)
print(f"Regression Coefficients: Eccentricity = {reg.coef_[0]:.3f}, Log(Flux) = {reg.coef_[1]:.3f}")
print(f"Intercept = {reg.intercept_:.3f}")

# Interpretation:
if abs(reg.coef_[1]) > abs(reg.coef_[0]):
    print("Stellar flux has a stronger effect on surface temperature than eccentricity.")
else:
    print("Orbital eccentricity plays a significant role in surface temperature variations.")


# Stellar Metallicity and Planet Formation: 

In [ ]:
df["S_METALLICITY"].fillna(df["S_METALLICITY"].mean(), inplace=True)
print(df["S_METALLICITY"].describe())

# Replace missing (NaN) values with the mean metallicity
df["S_METALLICITY"].fillna(df["S_METALLICITY"].mean(), inplace=True)

# Create a binary column for terrestrial planets
df["Is_Terrestrial"] = (df["P_TYPE"] == "Terran").astype(int)
print(df["Is_Terrestrial"].value_counts())

#print(df[df["Is_Terrestrial"] == 1]["S_METALLICITY"].isna().sum())
#print(df.groupby("Is_Terrestrial")["S_METALLICITY"].describe())

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df is your dataset containing "S_METALLICITY" and "Is_Terrestrial"
plt.figure(figsize=(8, 6))

# Histogram of metallicity for terrestrial vs. non-terrestrial planets
sns.histplot(df, x="S_METALLICITY", hue="Is_Terrestrial", bins=50, kde=True, element="step", common_norm=False, palette={0: "red", 1: "blue"})

plt.axvline(0.0, color="black", linestyle="--", label="Solar Metallicity (Sun)")

plt.xlabel("Host Star Metallicity")
plt.ylabel("Count")
plt.title("Distribution of Host Star Metallicity")
plt.legend(["Solar Metallicity", "Non-Terrestrial", "Terrestrial"])
plt.grid(True)
plt.show()

# Scatter plot: Metallicity vs. Terrestrial planet fraction
df_grouped = df.groupby("S_METALLICITY")["Is_Terrestrial"].mean().reset_index()

plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_grouped["S_METALLICITY"], y=df_grouped["Is_Terrestrial"], color="blue")
plt.axvline(0.0, color="black", linestyle="--", label="Solar Metallicity (Sun)")
plt.xlabel("Host Star Metallicity")
plt.ylabel("Fraction of Terrestrial Planets")
plt.title("Metallicity vs. Likelihood of Terrestrial Planets")
plt.legend()
plt.grid(True)
plt.show()




# Planetary Density and Core Composition:

# Semi-Major Axis and Surface Temperature:

In [ ]:
df["P_SEMI_MAJOR_AXIS"].fillna(df["P_SEMI_MAJOR_AXIS"].mean(), inplace=True)
df["P_TEMP_EQUIL"].fillna(df["P_TEMP_EQUIL"].mean(), inplace=True)

# Log transformation to better visualize the spread
df["log_SemiMajor"] = np.log10(df["P_SEMI_MAJOR_AXIS"])
df["log_Temp"] = np.log10(df["P_TEMP_EQUIL"])

# Scatter plot with habitability zone highlighted
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df["log_SemiMajor"], y=df["log_Temp"], alpha=0.6, color="blue", label="All Planets")

# Highlight planets in the habitable temperature range (200K - 300K)
habitable = df[(df["P_TEMP_EQUIL"] >= 200) & (df["P_TEMP_EQUIL"] <= 300)]
sns.scatterplot(x=habitable["log_SemiMajor"], y=habitable["log_Temp"], color="red", label="Habitable Zone")

plt.xlabel("log(Semi-Major Axis) [AU]")
plt.ylabel("log(Equilibrium Temperature) [K]")
plt.title("Orbital Distance vs. Equilibrium Temperature")
plt.legend()
plt.grid(True, linestyle="--")
plt.show()

# Percentage of planets in the habitable zone
habitable_percentage = (len(habitable) / len(df)) * 100
print(f"Percentage of planets in the habitable zone: {habitable_percentage:.2f}%")


# Mass vs. Orbital Period for Close-In Planets:

In [ ]:
df["P_PERIOD"].fillna(df["P_PERIOD"].mean(), inplace=True)
df["P_MASS"].fillna(df["P_MASS"].mean(), inplace=True)

# Tidal Locking Condition (Same as Previous)
tidally_locked = (df["P_PERIOD"] < 10) & (df["P_MASS"] < 10)  # Small planets + short period


# Scatter Plot: All Planets
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x=df["P_PERIOD"],
    y=df["P_MASS"],
    color="gray",  # Default color for all planets
    alpha=0.5,  # Light gray for background planets
    label="All Planets"
)

# Highlight Tidally Locked Planets
sns.scatterplot(
    x=df[tidally_locked]["P_PERIOD"],
    y=df[tidally_locked]["P_MASS"],
    color="red",  # Highlighted planets in red
    alpha=0.8,
    label="Tidally Locked"
)

plt.xscale("log")  # Log scale for better visualization
plt.yscale("log")
plt.xlabel("Orbital Period (Days)")
plt.ylabel("Mass (Earth Masses)")
plt.title("Tidal Locking: Mass vs. Orbital Period")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)

plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Constants
k = 1.38e-23  # Boltzmann constant (J/K)
m_avg = (18.015 + 32.00 + 28.014) / 3 / 1000 / 6.022e23  # Average molecular mass (kg)
threshold_habitable = 1  # Escape velocity must be 1.5x thermal velocity

# Load dataset
df = pd.read_csv("new.csv")

# Ensure valid values
df = df[(df["P_TEMP_EQUIL"] > 0) & (df["P_ESCAPE"] > 0)]

# Calculate Thermal Velocity (average gas)
df["V_thermal"] = np.sqrt((2 * k * df["P_TEMP_EQUIL"]) / m_avg) / 1000  # Convert to km/s

# Classify planets as habitable or not
df["Habitable"] = df["P_ESCAPE"] > df["V_thermal"] * threshold_habitable

# Count results
num_habitable = df["Habitable"].sum()
num_not_habitable = len(df) - num_habitable
print(f"Habitable Planets: {num_habitable}")
print(f"Not Habitable Planets: {num_not_habitable}")

# Scatter Plot: Escape Velocity vs. Thermal Velocity
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x="P_ESCAPE", y="V_thermal", hue="Habitable", palette={True: "green", False: "red"}, alpha=0.7)
plt.xlabel("Escape Velocity (km/s)")
plt.ylabel("Thermal Velocity (km/s)")
plt.title("Atmospheric Retention Based on Escape Velocity")
plt.legend()
plt.show()


In [ ]:
EARTH_MASS = 1.0  # Solar masses
EARTH_PERIOD = 365.25  # Days

# Define tolerance for similarity (adjustable based on scientific reasoning)
MASS_TOLERANCE = 0.1  # Acceptable deviation from Earth's mass
PERIOD_TOLERANCE = 50  # Acceptable deviation in days

# Identify habitable planets
df["Habitable"] = ((df["S_MASS"].between(EARTH_MASS - MASS_TOLERANCE, EARTH_MASS + MASS_TOLERANCE)) &
                     (df["P_PERIOD"].between(EARTH_PERIOD - PERIOD_TOLERANCE, EARTH_PERIOD + PERIOD_TOLERANCE)))

# Count the number of habitable planets
num_habitable = df["Habitable"].sum()
print(f"Number of habitable planets: {num_habitable}")

# Scatter plot highlighting habitable planets
plt.figure(figsize=(60, 6))
sns.scatterplot(data=df, x="S_MASS", y="P_PERIOD", hue="Habitable", palette={True: "blue", False: "gray"}, alpha=0.6, edgecolor=None)
plt.xlabel("Host Star Mass (Solar Masses)")
plt.ylabel("Orbital Period (days)")
plt.title("Comparison of Stellar Mass and Orbital Period to Earth's Conditions")
plt.legend(title="Habitable")
plt.show()

